This is an [jupyter](http://jupyter.org) notebook.
Lectures about Python, useful both for beginners and experts, can be found at http://scipy-lectures.github.io.

Open the notebook by (1) copying this file into a directory, (2) in that directory typing 
jupyter-notebook
and (3) selecting the notebook.

***
Written By: **Shashwat Shukla, Om Kolhe**
***

In this exercise, we will learn and code about Adaboost algorithm and look at one of it's application.

# Adaboost
## Motivation

Add content here

## Simulated Example

Explain the algorithm step by step and code after each step.